<a href="https://colab.research.google.com/github/cyyeh/kaggle/blob/master/google-qa/google_qa_yesno_albert_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Environment Setup

In [0]:
import json
import os
import pandas as pd

In [1]:
!pip install transformers # BertModel

     |████████████████████████████████| 481kB 7.0MB/s 
     |████████████████████████████████| 1.0MB 44.9MB/s 
     |████████████████████████████████| 3.1MB 38.8MB/s 
     |████████████████████████████████| 870kB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=7f3bebf47ed6d3a091c92390cfd9f4a712785b4c17c8ce7e483c483cd3baf792
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


### Install cuDF

In [2]:
!nvidia-smi

Mon Feb 17 14:23:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [4]:
!pip install cudf-cuda100

     |████████████████████████████████| 17.2MB 197kB/s 
     |████████████████████████████████| 12.4MB 36.3MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
     |████████████████████████████████| 9.1MB 27.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Found existing installation: numba 0.47.0
    Uninstalling numba-0.47.0:
      Successfully uninstalled numba-0.47.0


In [1]:
!find / -name librmm.so*

/usr/local/lib/python3.6/dist-packages/librmm.so


In [0]:
!cp /usr/local/lib/python3.6/dist-packages/librmm.so .

In [0]:
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

In [0]:
import nvstrings, nvcategory, cudf

# Prepare YES/NO Answer Dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Check if training/testing dataset is available in your google drive. If it's not available, you should run code inside the "Prepare Kaggle Dataset" section.

In [0]:
if os.path.exists('drive/My Drive/yes_no_ans_df.pkl') and \
os.path.exists('drive/My Drive/short_ans_raw_df.pkl'):
  print("Training dataset is available!")
else:
  print("Training dataset is not found, please run code inside the 'Prepare Kaggle Dataset' section.")

Training dataset is available!


## Prepare Kaggle Dataset for [TensorFlow 2.0 Question Answering](https://www.kaggle.com/c/tensorflow2-question-answering)

### Download Question Answering Dataset

In [7]:
import os
os.environ['KAGGLE_USERNAME'] = "chihyuyeh" # username from the json file
os.environ['KAGGLE_KEY'] = "f21b340fc8082977cbf954c80ad69ae1" # key from the json file
!kaggle competitions download -c tensorflow2-question-answering

100% 4.78M/4.78M [00:00<00:00, 25.2MB/s]

  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 17.7MB/s]
100% 4.45G/4.46G [01:30<00:00, 61.3MB/s]
100% 4.46G/4.46G [01:30<00:00, 52.8MB/s]


In [8]:
!unzip simplified-nq-train.jsonl.zip
!unzip simplified-nq-test.jsonl.zip

Archive:  simplified-nq-train.jsonl.zip
  inflating: simplified-nq-train.jsonl  
Archive:  simplified-nq-test.jsonl.zip
  inflating: simplified-nq-test.jsonl  


### Generate Short Answer Raw Data Dataframe

In [0]:
train_path = 'simplified-nq-train.jsonl'
test_path = 'simplified-nq-test.jsonl'

In [0]:
def extracting_text_using_start_end_token_id(document_text, start_token, end_token):
    splitted_document_text = document_text.split()
    return ' '.join(splitted_document_text[start_token:end_token])

In [0]:
def has_long_answer(long_answer_candidate):
  return long_answer_candidate['start_token'] != -1 \
  and long_answer_candidate['candidate_index'] != -1 \
  and long_answer_candidate['end_token'] != -1

In [0]:
def data_cleaning_for_short_answer(
  json_obj,
  task='both',
  example_id=True):
  '''
  keys of the output dictionary: 
    'example_id' # optional 
    'question_text'
    'long_answer_text'
    'yes_no_answer' # exist only if task == 'both' or 'classing'
    'short_answer_start_token' # exist only  if task == 'both' or 'squading'
    'short_answer_end_token' # exist only if task == 'both' or 'squading'
  ''' 
  assert task == 'classing' or task == 'squading' or task == 'both'
  new_data_d = {}
  # assignment for both tasks  
  annotations = json_obj['annotations'][0]
  long_answer_candidate = annotations['long_answer']
  if example_id:
    new_data_d['example_id'] = json_obj['example_id']
  new_data_d['question_text'] = json_obj['question_text']
  long_ans_start = long_answer_candidate['start_token']
  long_ans_end = long_answer_candidate['end_token']
  new_data_d['long_answer_text'] = (
    extracting_text_using_start_end_token_id(
      json_obj['document_text'],
      long_ans_start,
      long_ans_end
    ))
  if task != 'both':
    if task == 'squading':
      short_answer_candidate = annotations['short_answers']
      if not short_answer_candidate:
        short_ans_start = -1
        short_ans_end = -1
      else:
        short_ans_start = short_answer_candidate[0]['start_token'] - long_ans_start
        short_ans_end = short_answer_candidate[0]['end_token'] - long_ans_start
      new_data_d['short_answer_start_token'] = short_ans_start
      new_data_d['short_answer_end_token'] = short_ans_end
    elif task == 'classing':
      new_data_d['yes_no_answer'] = annotations['yes_no_answer']
  else:
    # get squading labels 
    short_answer_candidate = annotations['short_answers']
    if not short_answer_candidate:
      short_ans_start = -1
      short_ans_end = -1
    else:
      short_ans_start = short_answer_candidate[0]['start_token'] - long_ans_start
      short_ans_end = short_answer_candidate[0]['end_token'] - long_ans_start
    new_data_d['short_answer_start_token'] = short_ans_start
    new_data_d['short_answer_end_token'] = short_ans_end
    # get classing labels 
    new_data_d['yes_no_answer'] = annotations['yes_no_answer']
  return new_data_d 

In [0]:
def create_short_answer_dataset(path):
  short_answer_dataset = []
  with open(path) as f:
    for line in f:
      old_data_d = json.loads(line)
      if has_long_answer(old_data_d['annotations'][0]['long_answer']):
        new_data_d = data_cleaning_for_short_answer(old_data_d)
        short_answer_dataset.append(new_data_d)
  return df.DataFrame(short_answer_dataset)

In [23]:
raw_df = create_short_answer_dataset(train_path)

print(len(raw_df))
raw_df.head()

ValueError: ignored

In [0]:
print(len(raw_df))
raw_df.head()

### Save Short Answer Dataset to Pickle Format and Export It To Google Drive

In [0]:
raw_df.to_pickle("./short_ans_raw_df.pkl")

In [0]:
!cp ./short_ans_raw_df.pkl /content/drive/My\ Drive

### Create YES/NO Answer Dataset

In [0]:
def create_short_ans_features(raw_df, mode='1'):
  '''
  parameters:
  raw_df: short answer dataframe
  mode: 1(default): yes/no answer; 2: short answer entity
  returns:
  dataframe for tokenizedshort answer dataset
  '''
  from transformers import AlbertTokenizer

  tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
  short_ans_features_dict = {
    'token_ids': [],
    'segment_ids': [],
    'mask_ids': []
  }

  if mode == '1':
    short_ans_features_dict['label_yes_no'] = []
  elif mode == '2':
    short_ans_features_dict['label_start_tokens'] = []
    short_ans_features_dict['label_end_tokens'] = []

  label_yes_no_map = {
    'YES': 0,
    'NO': 1,
    'NONE': 2
  }

  MAX_LENGTH = 512

  for _, row in raw_df.iterrows():
    # tokenize question text
    tokens = ['[CLS]'] + tokenizer.tokenize(row['question_text']) + ['[SEP]']
    sentence_A_len = len(tokens)
  
    # tokenize no long answer
    if row['short_answer_start_token'] == -1:
      tokens = tokens + tokenizer.tokenize(row['long_answer_text']) + ['[SEP]']
      sentence_len = len(tokens)
      label_start_token = 0
      label_end_token = 0
    # tokenize short answer span
    else:
      # cut long answer into 3 chunks
      long_answer = row['long_answer_text'].split()
      chunk_1 = ' '.join(long_answer[:row['short_answer_start_token']])
      chunk_2 = ' '.join(long_answer[row['short_answer_start_token']:row['short_answer_end_token']])
      chunk_3 = ' '.join(long_answer[row['short_answer_end_token']:])

      # handle new start end token
      tokens = tokens + tokenizer.tokenize(chunk_1)
      label_start_token = len(tokens)
      tokens = tokens + tokenizer.tokenize(chunk_2)
      label_end_token = len(tokens)

      tokens = tokens + tokenizer.tokenize(chunk_3) + ['[SEP]']
      sentence_len = len(tokens)

    # apply truncating
    if sentence_len > MAX_LENGTH:
      tokens = tokens[:MAX_LENGTH-1] + ['[SEP]']
      sentence_len = MAX_LENGTH
    if label_end_token > MAX_LENGTH - 1: # should not exceed last token [SEP]
      label_start_token = 0
      label_end_token = 0

    # create segment_id and mask_id
    segment_ids = sentence_A_len * [0] + (sentence_len - sentence_A_len) * [1] 
    mask_ids = sentence_len * [1]

    # apply padding
    if (sentence_len < MAX_LENGTH):
      pad_len = MAX_LENGTH - sentence_len
      tokens = tokens + pad_len * ['[PAD]']
      segment_ids = segment_ids + pad_len * [0]
      mask_ids = mask_ids + pad_len * [0]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # yes_no_label
    yes_no_label = label_yes_no_map[row['yes_no_answer']]

    # append to lists
    short_ans_features_dict['token_ids'].append(token_ids)
    short_ans_features_dict['segment_ids'].append(segment_ids)
    short_ans_features_dict['mask_ids'].append(mask_ids)
    if mode == '1':
      short_ans_features_dict['label_yes_no'].append(yes_no_label)
    elif mode == '2':
      short_ans_features_dict['label_start_tokens'].append(label_start_token)
      short_ans_features_dict['label_end_tokens'].append(label_end_token)

  return pd.DataFrame(short_ans_features_dict)

In [124]:
yes_no_ans_df = create_short_ans_features(raw_df)
print(len(yes_no_ans_df))
yes_no_ans_df.head()

152148


,token_ids,segment_ids,mask_ids,label_yes_no
0,"[2, 56, 25, 14, 127, 757, 275, 16, 17034, 8, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
1,"[2, 184, 31, 9, 5909, 154, 449, 72, 25, 14, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
2,"[2, 98, 1001, 16, 4270, 8005, 4330, 1384, 209,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,"[2, 72, 41, 14, 127, 4041, 19, 14, 4101, 3, 13...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
4,"[2, 72, 257, 169, 3409, 16931, 16, 14, 9358, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


### Save YES/NO Answer Dataset to Pickle Format and Export It To Google Drive

In [0]:
# save dataframe to pickle file
yes_no_ans_df.to_pickle("./yes_no_ans_df.pkl")

In [0]:
!cp ./yes_no_ans_df.pkl /content/drive/My\ Drive